In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style('ticks')

In [2]:
from nltk import ngrams
from collections import defaultdict
from gensim.models import Word2Vec

In [ ]:
#########################

In [87]:
for i in range(5):
    knn_model = KNN(n_neighbors = 10 + i*5)
    knn_model.fit(train[w2v_columns], train['Energ_Kcal'])
    temp = knn_model.predict(train[w2v_columns])
    train['knn_{}'.format(5 + i*5)] = temp

In [86]:
from sklearn.neighbors import KNeighborsRegressor as KNN

In [14]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression

In [88]:
all_names = train.columns.values

In [89]:
class full_model:
    def __init__(self):
        self.xgb = xgb.XGBRegressor(silent=True)
        self.lin = LinearRegression()
        #self.lin2 = LinearRegression()
        
    def fit(self, X, y):
        self.xgb.fit(X, y, eval_metric = mean_absolute_error)
        self.lin.fit(X, y)
        #self.lin2.fit(np.concatenate((self.xgb.predict(X).reshape(-1, 1),
        #            self.lin.predict(X).reshape(-1, 1)), axis=1), y)
        
    def predict(self, X):
        #return self.lin2.predict(np.concatenate((self.xgb.predict(X).reshape(-1, 1),
        #            self.lin.predict(X).reshape(-1, 1)), axis=1))
        return 0.5 * (self.xgb.predict(X) + self.lin.predict(X))

In [90]:
train = train.fillna(0)

In [91]:
def trainXGB(names_list):
    X_train, X_test, y_train, y_test = train_test_split(
        train[names_list].drop(['Energ_Kcal', 'Full_descr'],
            axis=1), train['Energ_Kcal'], test_size=0.1, random_state=42)
    
    kf = KFold(n_splits=5, random_state=None, shuffle=True)
    ans = []
    ans2 = []
    for train_index, val_index in kf.split(X_train):
        model = full_model()
        X_ktrain, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_ktrain, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        model.fit(X_ktrain.values, y_ktrain.values)
        ans += [mean_absolute_error(model.predict(X_test.values), y_test)]
        ans2 += [mean_absolute_error(model.predict(X_val.values), y_val)]
        
        #
        #
        #break
        
    return np.array(ans), np.array(ans2)

In [92]:
all_res, all_res_val = trainXGB(all_names)
all_res

array([6.15149628, 6.3127459 , 6.25126594, 6.07384851, 6.39481496])

In [93]:
all_res_val

array([6.13274776, 6.16907667, 5.99187875, 6.1507376 , 5.80413142])

In [189]:
#############################

In [10]:
# data import
train = pd.read_excel('semifinal_data/train.xlsx')
test = pd.read_excel('semifinal_data/test.xlsx')
descr = pd.read_excel('semifinal_data/description.xlsx')

In [11]:
# word info gathering
train['Full_descr'] = train['Shrt_Desc'].values + ' ' +\
    train['GmWt_Desc1'].fillna('').values + ' ' +\
    train['GmWt_Desc2'].fillna('').values

train = train.drop(['Shrt_Desc', 'GmWt_Desc1', 'GmWt_Desc2'], axis = 1)

In [12]:
def prepare_description(description):
    word_form = description.lower().split()
        
    res = []
    for i in word_form:
        res += i.split(',')
        
    for i in range(len(res)):
        try_to_find = res[i].find('w/')
        if (try_to_find != -1):
            res[i] = res[i][:try_to_find] + res[i][try_to_find + 2:]
        
    return res

# lowering and deleting 'w/' substrings
word_information = train['Full_descr'].apply(prepare_description)

In [13]:
text = []
for i in range(1, 14):
    with open("semifinal_files/book_{}.txt".format(i), 'r') as file:
        text += [eval(file.read())]

# lowering and getting rid of short useless words, 
# which sometimes appear in this dataset
for i in range(len(text)):
    j = 0
    while j < len(text[i]):
        text[i][j] = text[i][j].lower()
        
        if len(text[i][j]) == 1:
            text[i].pop(j)
            j -= 1
        j += 1

In [14]:
cook_book = Word2Vec(text, min_count=1, workers=5,
                 window=4, sg = 2, iter = 5)

In [15]:
all_words = set()
for item in cook_book.wv.vocab:
    all_words.add(item)

Каждому типу продукта сопоставим слова из доступного словаря:\
1) Хлебобулочные изделия : 'bakery', 'bread', 'loaf', 'bun'\
2) Жидкость : 'liquid', 'water', 'juice', 'wine', 'cocktail'\
3) Молочная продукция : 'milk', 'dairy', 'yogurt', 'buttermilk'\
4) Мясная продукция : 'meat', 'lamb', 'pork', 'mutton'\
5) Овощи / фрукты : 'fruit', 'vegetable', 'apple', 'banana', 'tomato', 'cucumber'\

In [16]:
def get_voting(description, key_words, min_confidence = 0.1):
    votes = 0
    sum_confidence = 0
    for word in description:
        if word in all_words:
            for key_word in key_words:
                sim = cook_book.wv.similarity(word, key_word)
                if sim >= min_confidence: 
                    sum_confidence += sim
                    votes += 1
    if not votes:
        return 0
    return sum_confidence / votes

In [18]:
label = [[]]*6
label[1] = ['bakery', 'bread', 'loaf', 'bun', 'sugar']
label[2] = ['liquid', 'water', 'juice', 'wine', 'cocktail']
label[3] = ['milk', 'dairy', 'yogurt', 'buttermilk']
label[4] = ['meat', 'lamb', 'pork', 'mutton', 'bone', 'raw']
label[5] = ['fruit', 'vegetable', 'salad', 'exotic']

for i in range(1, 6):
    train['label_{}'.format(i)] = word_information.apply(lambda x : get_voting(x, label[i], 0.2))

In [21]:
train

,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),Calcium_(mg),Iron_(mg),...,Cholestrl_(mg),GmWt_1,GmWt_2,Refuse_Pct,Full_descr,label_1,label_2,label_3,label_4,label_5
0,4.70,393,12.00,6.00,3.90,73.40,5.2,13.75,651.0,47.50,...,0.0,2.0,15.0,0.0,"BABYFOOD,CRL,OATMEAL,W/BANANAS,DRY 1 tbsp .5 oz",0.903862,0.882051,0.891199,0.904462,0.835817
1,90.20,28,2.52,0.62,2.04,4.62,2.4,NaN,58.0,3.13,...,0.0,146.0,33.0,0.0,"HEARTS OF PALM,CANNED 1 cup 1 piece",0.917030,0.887706,0.912919,0.898939,0.952904
2,63.89,196,24.06,11.09,1.56,0.00,0.0,0.00,13.0,0.97,...,130.0,89.0,85.0,29.0,"CHICKEN,BROILERS OR FRYERS,THIGH,MEAT ONLY,CKD...",0.878424,0.851435,0.852720,0.907550,0.924382
3,54.73,280,20.43,21.35,3.51,0.00,0.0,0.00,10.0,0.95,...,58.0,140.0,85.0,27.0,"PORK,CURED,SHLDR,ARM PICNIC,LN&FAT,RSTD 1 cup ...",0.945092,0.909155,0.941600,0.935186,0.936590
4,94.78,18,0.00,0.00,0.02,0.04,0.0,0.04,6.0,0.03,...,0.0,14.0,238.0,0.0,"VINEGAR,DISTILLED 1 tbsp 1 cup",0.924237,0.898129,0.941379,0.853670,0.945844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7751,77.33,119,17.54,4.93,1.08,0.00,0.0,NaN,60.0,0.90,...,64.0,85.0,198.0,0.0,"DRUM,FRESHWATER,RAW 3 oz 1 fillet",0.944371,0.902569,0.939408,0.933046,0.911712
7752,63.95,224,20.59,15.04,0.87,0.00,0.0,0.00,23.0,1.47,...,80.0,85.0,NaN,9.0,"BEEF,LOIN,TOP LOIN,LN & FAT,1/8"" FAT,SEL,RAW 3...",0.889046,0.846518,0.867114,0.934771,0.908709
7753,74.00,132,5.06,7.59,2.09,11.27,1.8,NaN,119.0,0.64,...,15.0,245.0,NaN,0.0,"POTATOES,AU GRATIN,HOME-PREPARED FROM RECIPE U...",0.891210,0.846278,0.832915,0.916679,0.899715
7754,79.36,74,0.28,0.12,0.33,19.91,2.6,NaN,4.0,0.43,...,0.0,193.0,NaN,0.0,"APPLES,DEHYD (LO MOIST),SULFURED,STWD 1 cup",0.944440,0.896799,0.948496,0.929006,0.986852


In [19]:
label_columns = ['label_{}'.format(i) for i in range(1, 6)]

In [20]:
train[label_columns].apply(lambda x : x.argmax(), axis=1)

/home/data_sapiens/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


0       label_4
1       label_5
2       label_5
3       label_1
4       label_5
         ...   
7751    label_1
7752    label_4
7753    label_4
7754    label_5
7755    label_5
Length: 7756, dtype: object

In [233]:
train['Full_descr'].iloc[-1]

'TUNA,FRESH,YELLOWFIN,RAW 1 oz, boneless 1 cubic inch, boneless'

In [151]:
def count_includence(x):
    res = 0
    for i in x:
        if i in all_words:
            res += 1
    return res / len(x)

In [188]:
(train['Full_descr'].apply(prepare_word).apply(count_includence) > 0.2).sum() / train.shape[0]

0.9573233625580196

In [ ]:
train['']